<a href="https://colab.research.google.com/github/nitinsayshe/Blog_Project/blob/project%2Fblog/T5_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -r requirements.txt

In [2]:
!python --version

Python 3.10.12


In [8]:
!pip install chromadb

In [4]:
import os
import chromadb
from chromadb.config import Settings
CHROMA_SETTINGS = Settings(
        chroma_db_impl='duckdb+parquet',
        persist_directory='db',
        anonymized_telemetry=False
)

In [9]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import os
from google.colab import files
persist_directory = "db"
# import chromadb
# from chromadb.config import Settings
# persistent_client = chromadb.PersistentClient()

for root, dirs, files in os.walk("docs"):
        for file in files:
            if file.endswith(".pdf"):
                print(file)
                loader = PyPDFLoader(os.path.join(root, file))

documents = loader.load()
print("splitting into chunks")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
#create embeddings here
print("Loading sentence transformers model")
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
#create vector store here
print(f"Creating embeddings. May take some minutes...")
db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory, client_settings=CHROMA_SETTINGS)
db.persist()
db=None

print(f"Ingestion complete!")

finance.pdf
splitting into chunks
Loading sentence transformers model
Creating embeddings. May take some minutes...
Ingestion complete!


In [10]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch

In [ ]:
!pip list

In [11]:
device = torch.device('cpu')
# checkpoint = "MBZUAI/LaMini-T5-738M"
checkpoint = "MBZUAI/LaMini-Flan-T5-783M"
print(f"Checkpoint path: {checkpoint}")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    device_map='auto',
    torch_dtype=torch.float32
)

Checkpoint path: MBZUAI/LaMini-Flan-T5-783M


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:


def llm_pipeline():
    pipe = pipeline(
        'text2text-generation',
        model = base_model,
        tokenizer = tokenizer,
        max_length = 256,
        do_sample = True,
        temperature = 0.3,
        top_p= 0.95,
        device=device
    )
    local_llm = HuggingFacePipeline(pipeline=pipe)
    return local_llm

def qa_llm():
    llm = llm_pipeline()
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    db = Chroma(persist_directory="db", embedding_function = embeddings, client_settings=CHROMA_SETTINGS)
    retriever = db.as_retriever()
    qa = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type = "stuff",
        retriever = retriever,
        return_source_documents=True
    )
    return qa

def process_answer(instruction):
    response = ''
    instruction = instruction
    qa = qa_llm()
    generated_text = qa(instruction)
    answer = generated_text['result']
    return answer, generated_text

In [13]:
question = "explain finance commsiion ?"
answer, metadata = process_answer(question)
print("ans", answer)

ans The Finance Commission of India is responsible for recommending the distribution of net proceeds of taxes to the President of India. The commission is responsible for ensuring that the principles that govern the grant-in-aid by the central government to the state government are out of the consolidated fund.


In [14]:
question = "what are the functions of finance commission of india and list them?"
answer, metadata = process_answer(question)
print("ans", answer)

ans The functions of Finance Commission of India are: 1. The distribution of the net proceeds of taxes should be shared among the centre and the states. 2. The principles that govern the grant-in-aid by the central government to the state government are out of the consolidated fund.
